In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 100  # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

## Heuristic - N1

In [7]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_50-100.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [06:10<00:00,  2.70it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   1.749760e+12               0.0      0.368375
std    9.192323e+12               0.0      0.154660
min    5.592422e+02               0.0      0.217728
25%    7.800283e+02               0.0      0.293457
50%    8.765984e+02               0.0      0.324888
75%    9.898740e+02               0.0      0.375477
max    4.999513e+13               0.0      1.483054
Number of infeasible solution: 0


## Learnable Rounding

In [9]:
# hyperparameters
penalty_weight = 1000 # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [10]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd])
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks)

In [11]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup)
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 14252.06
Epoch 1, Validation Loss: 18579.37
Epoch 2, Validation Loss: 1705.47
Epoch 3, Validation Loss: 1314.75
Epoch 4, Validation Loss: 1045.25
Epoch 5, Validation Loss: 1087.64
Epoch 6, Validation Loss: 2296.91
Epoch 7, Validation Loss: 1062.06
Epoch 8, Validation Loss: 1284.42
Epoch 9, Validation Loss: 742.38
Epoch 10, Validation Loss: 724.75
Epoch 11, Validation Loss: 724.58
Epoch 12, Validation Loss: 1533.65
Epoch 13, Validation Loss: 611.55
Epoch 14, Validation Loss: 838.61
Epoch 15, Validation Loss: 842.80
Epoch 16, Validation Loss: 726.72
Epoch 17, Validation Loss: 684.70
Epoch 18, Validation Loss: 712.34
Epoch 19, Validation Loss: 680.49
Epoch 20, Validation Loss: 739.94
Epoch 21, Validation Loss: 752.61
Epoch 22, Validation Loss: 987.68
Epoch 23, Validation Loss: 707.38
Epoch 24, Validation Loss: 696.95
Epoch 25, Validation Loss: 725.75
Epoch 26, Validation Loss: 630.35
Epoch 27, Validation Loss: 593.27
Epoch 28, Validation Loss: 672.58
Epoch 29, Va

In [12]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lr_50-100.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 156.48it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    591.572404          0.009632      0.001098
std     224.137861          0.194556      0.000453
min     263.535047          0.000000      0.000000
25%     398.548331          0.000000      0.001000
50%     566.494457          0.000000      0.001001
75%     752.265120          0.000000      0.001012
max    1641.545864          5.298768      0.002639
Number of infeasible solution: 4


## Learnable Threshold

In [13]:
# hyperparameters
penalty_weight = 1000 # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd])
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup)
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 11967.23
Epoch 1, Validation Loss: 9519.18
Epoch 2, Validation Loss: 1061.94
Epoch 3, Validation Loss: 1005.62
Epoch 4, Validation Loss: 887.19
Epoch 5, Validation Loss: 816.39
Epoch 6, Validation Loss: 1274.08
Epoch 7, Validation Loss: 1320.71
Epoch 8, Validation Loss: 768.93
Epoch 9, Validation Loss: 754.42
Epoch 10, Validation Loss: 691.60
Epoch 11, Validation Loss: 760.88
Epoch 12, Validation Loss: 754.72
Epoch 13, Validation Loss: 1886.83
Epoch 14, Validation Loss: 207339.47
Epoch 15, Validation Loss: 718504.08
Epoch 16, Validation Loss: 767.73
Epoch 17, Validation Loss: 710.53
Epoch 18, Validation Loss: 12303439.10
Epoch 19, Validation Loss: 567408.22
Epoch 20, Validation Loss: 851.06
Epoch 21, Validation Loss: 723.20
Epoch 22, Validation Loss: 758.00
Epoch 23, Validation Loss: 786.17
Epoch 24, Validation Loss: 702.97
Epoch 25, Validation Loss: 671.91
Epoch 26, Validation Loss: 746.69
Epoch 27, Validation Loss: 14755.37
Epoch 28, Validation Loss: 722.34


In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lt_50-100.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 152.04it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    624.494589          0.005877      0.001186
std     219.343950          0.185842      0.000494
min     287.469959          0.000000      0.000000
25%     463.392384          0.000000      0.001000
50%     566.782938          0.000000      0.001003
75%     738.285202          0.000000      0.001472
max    1864.226095          5.876848      0.004025
Number of infeasible solution: 1
